# Деревья поиска

In [7]:
#include <iostream>
#include <stack>

using namespace std;

## Дерево поиска

классическое дерево поиска

In [1]:
struct SearchTreeNode {
    int key;
    int data;
    SearchTreeNode *left, *right;
};

In [2]:
class SearchTree {
private:

    SearchTreeNode *root;
    int count;
    int invalid;
public:
    SearchTree(int invalidData);
    ~SearchTree();

    void print();
    int minKeyData();
    int maxKeyData();
    int getDataByKey(int key);
    void insert(int key, int data);
    void deleteNode(int key);
};

In [3]:
SearchTreeNode *newNodeSearch(int item, int data) {
    auto *temp = new SearchTreeNode;
    temp->key = item;
    temp->data = data;
    temp->left = temp->right = nullptr;
    return temp;
}


In [4]:
SearchTree::SearchTree(int invalidData) {
    root = nullptr;
    count = 0;
    invalid = invalidData;
}

In [8]:
SearchTree::~SearchTree() {
    count = 0;

    stack<SearchTreeNode*> stack;
    stack.push(root);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }

}

In [9]:
void inorder(SearchTreeNode *root) {
    if (root != nullptr) {
        inorder(root->left);
        cout << root->key << " -> ";
        inorder(root->right);
    }
}

In [10]:
void SearchTree::print() {
    inorder(root);
}

In [11]:
SearchTreeNode* leftStep(SearchTreeNode* root){
    if (root != nullptr)
        return root->left;
    return nullptr;
}

In [12]:
int SearchTree::minKeyData(){
    auto tmp = root;
    if (tmp == nullptr)
        return invalid;
    while (tmp != nullptr)
        tmp = leftStep(tmp);
    return tmp->data;
}

In [13]:
SearchTreeNode* rightStep(SearchTreeNode* root){
    if (root != nullptr)
        return root->right;
    return nullptr;
}

In [14]:
int SearchTree::maxKeyData(){
    auto tmp = root;
    if (tmp == nullptr)
        return invalid;
    while (tmp != nullptr)
        tmp = rightStep(tmp);
    return tmp->data;
}

In [15]:
SearchTreeNode* search(SearchTreeNode* root, int key){
    if (root == nullptr || root->key == key)
        return root;
    if (root->key < key)
        return search(root->right, key);
    return search(root->left, key);
}

In [16]:
int SearchTree::getDataByKey(int key){
    auto node = search(root, key);
    if (node != nullptr)
        return node->data;
    return invalid;
}

In [17]:
SearchTreeNode* insert(SearchTreeNode *node, int key, int data) {
    if (node == nullptr) return newNodeSearch(key, data);

    if (key < node->key)
        node->left = insert(node->left, key, data);
    else
        node->right = insert(node->right, key, data);

    return node;
}

In [18]:
void SearchTree::insert(int key, int data){
    root = ::insert(root, key, data);
}

In [19]:
SearchTreeNode* searchParent(SearchTreeNode* root, int key){
    if (root == nullptr)
        return root;
    if ((root->left != nullptr) && (root->left->data == key))
        return  root;
    if ((root->right != nullptr) && (root->right->data == key))
        return  root;
    if (root->key < key)
        return search(root->right, key);
    return search(root->left, key);
}

In [20]:
SearchTreeNode* deleteNode(SearchTreeNode* root, int key){
    if (root == nullptr) return root;

    if (key < root->key)
        root->left = deleteNode(root->left, key);
    else if (key > root->key)
        root->right = deleteNode(root->right, key);
    else {
        if (root->left == nullptr) {
            auto temp = root->right;
            delete root;
            return temp;
        } else if (root->right == nullptr) {
            auto  *temp = root->left;
            delete root;
            return temp;
        }

        //если узел в корне надо в правом дереве найти минимум и поставить в корень
        auto prnt = root;
        auto curr = root->right;
        while (curr->left != nullptr) {
            prnt = curr;
            curr = curr->left;
        }
        root->key = curr->key;
        root->right = deleteNode(root->right, curr->key);
    }
    return root;
}

In [21]:
void SearchTree::deleteNode(int key) {
    root = ::deleteNode(root, key);
}

## AVL-дерево

### Концепция поворотов

реализация

Концепция: разность высот 2-х поддеревьев не больше чем 1.

In [22]:
class AVLTreeNode {
public:
    int key;
    int data;
    AVLTreeNode *left;
    AVLTreeNode *right;
    int height;
};

In [23]:
class AVLTree {
private:
    AVLTreeNode *root;
    int count;
    int invalid;
public:
    AVLTree(int invalid);
    ~AVLTree();

    void print();
    int minKeyData();
    int maxKeyData();
    int getDataByKey(int key);
    void insert(int key, int data);
    void deleteNode(int key);
};

In [24]:
#include <iostream>
#include <stack>
#include <string>
using namespace std;

In [25]:
AVLTreeNode *newNode(int key, int data) {
    auto node = new AVLTreeNode;
    node->key = key;
    node->data = data;
    node->left = nullptr;
    node->right = nullptr;
    node->height = 1;
    return node;
}

In [26]:
int max(int a, int b) {
    return (a > b) ? a : b;
}

In [27]:
int height(AVLTreeNode *N) {
    if (N == nullptr)
        return 0;
    return N->height;
}

In [28]:
AVLTreeNode *rightRotate(AVLTreeNode *y) {
    auto x = y->left;
    auto t2 = x->right;
    x->right = y;
    y->left = t2;
    y->height = max(height(y->left),height(y->right)) + 1;
    x->height = max(height(x->left),height(x->right)) + 1;
    return x;
}

In [29]:
AVLTreeNode *leftRotate(AVLTreeNode *x) {
    auto y = x->right;
    auto T2 = y->left;
    y->left = x;
    x->right = T2;
    x->height = max(height(x->left),height(x->right)) + 1;
    y->height = max(height(y->left),height(y->right)) + 1;
    return y;
}

In [30]:
int getBalanceFactor(AVLTreeNode *node) {
    if (node == nullptr)
        return 0;
    return height(node->left) - height(node->right);
}

In [31]:
AVLTreeNode *minNode(AVLTreeNode *node) {
    AVLTreeNode *current = node;
    while (current->left != nullptr)
        current = current->left;
    return current;
}

In [32]:
AVLTree::AVLTree(int invalid){
    root = nullptr;
    count = 0;
    this->invalid = invalid;
}

In [33]:
AVLTree::~AVLTree(){
    count = 0;

    stack<AVLTreeNode*> stack;
    stack.push(root);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }
}

In [34]:
void printTree(AVLTreeNode *root, string indent, bool last) {
    if (root != nullptr) {
        cout << indent;
        if (last) {
            cout << "R----";
            indent += "   ";
        } else {
            cout << "L----";
            indent += "|  ";
        }
        cout << root->key << endl;
        printTree(root->left, indent, false);
        printTree(root->right, indent, true);
    }
}

In [35]:
void AVLTree::print(){
    printTree(root, "", true);
}

In [36]:
int AVLTree::minKeyData(){
    auto curr = root;
    while (curr->left != nullptr) {
        curr = curr->left;
    }
    return curr->data;
}

In [37]:
int AVLTree::maxKeyData(){
    auto curr = root;
    while (curr->right != nullptr) {
        curr = curr->right;
    }
    return curr->data;
}

In [38]:
AVLTreeNode* search(AVLTreeNode* root, int key){
    if (root == nullptr || root->key == key)
        return root;
    if (root->key < key)
        return search(root->right, key);
    return search(root->left, key);
}

In [39]:
int AVLTree::getDataByKey(int key){
    auto node = search(root, key);
    if (node != nullptr)
        return node->data;
    return invalid;
}

In [40]:
AVLTreeNode *insertNode(AVLTreeNode *node, int key, int data) {

    if (node == nullptr)
        return (newNode(key, data));
    if (key < node->key)
        node->left = insertNode(node->left, key,data);
    else if (key > node->key)
        node->right = insertNode(node->right, key, data);
    else {
        node->data = data;
        return node;
    }

    node->height = 1 + max(height(node->left),height(node->right));
    int balanceFactor = getBalanceFactor(node);
    if (balanceFactor > 1) {
        if (key < node->left->key) {
            return rightRotate(node);
        } else if (key > node->left->key) {
            node->left = leftRotate(node->left);
            return rightRotate(node);
        }
    }
    if (balanceFactor < -1) {
        if (key > node->right->key) {
            return leftRotate(node);
        } else if (key < node->right->key) {
            node->right = rightRotate(node->right);
            return leftRotate(node);
        }
    }
    return node;
}

In [41]:
void AVLTree::insert(int key, int data){
    root = insertNode(root, key, data);
}

In [42]:
AVLTreeNode *deleteNode(AVLTreeNode *root, int key) {

    if (root == nullptr)
        return root;
    if (key < root->key)
        root->left = deleteNode(root->left, key);
    else if (key > root->key)
        root->right = deleteNode(root->right, key);
    else {
        if ((root->left == nullptr) ||
            (root->right == nullptr)) {
            auto temp = root->left ? root->left : root->right;
            if (temp == nullptr) {
                temp = root;
                root = nullptr;
            } else
                *root = *temp;
            delete temp;
        } else {
            auto temp = minNode(root->right);
            root->key = temp->key;
            root->right = deleteNode(root->right,temp->key);
        }
    }
    if (root == nullptr)
        return root;

    root->height = 1 + max(height(root->left),height(root->right));
    int balanceFactor = getBalanceFactor(root);
    if (balanceFactor > 1) {
        if (getBalanceFactor(root->left) >= 0) {
            return rightRotate(root);
        } else {
            root->left = leftRotate(root->left);
            return rightRotate(root);
        }
    }
    if (balanceFactor < -1) {
        if (getBalanceFactor(root->right) <= 0) {
            return leftRotate(root);
        } else {
            root->right = rightRotate(root->right);
            return leftRotate(root);
        }
    }
    return root;
}

In [43]:
void AVLTree::deleteNode(int key){
    root = ::deleteNode(root, key);
}

## Красно-черное дерево

Свойства:
1. Каждый узел красный или черный
2. Корень и листья - черные
3. У красного узла родительский узел - черный
4. Все простые пути из любого узла Х к листьям содержат одинаковое кол-во черных узлов

Будем называть чёрной высотой (англ. black-height) вершины x число чёрных вершин на пути из x в лист.

Лемма. В КЧД с черной высотой b не менее $2^b - 1$ узлов

Докажем по индукции по обычной высоте h(x), что поддерево любого узла x с черной высотой hb(x) содержит не менее $2^{hb(x−1)}−1$ внутренних узлов. Здесь h(x) — кратчайшее расстояние от вершины x до какого-то из листьев. 

База индукции:

Если высота узла x
равна 1, то x — это лист, hb(x)=1, $2^{1−1}−1=0$. 

Переход:
Так как любая внутренняя вершина (вершина, у которой высота положительна) имеет двух потомков, то применим предположение индукции к ним — их высоты на единицу меньше высоты x. Тогда черные высоты детей могут быть hb(x) или hb(x)−1 — если потомок красный или черный соответственно. 
Тогда по предположению индукции в каждом из поддеревьев не менее $2^{hb(x)−2}−1$ вершин. Тогда всего в поддереве не менее $2(2^{hb(x)−2}−1)+1=2^{hb(x)−1}−1$ вершин (+1 — мы учли еще саму вершину x). 
Переход доказан. Теперь, если мы рассмотрим корень всего дерева в качестве x, то получится, что всего вершин в дереве не менее $2^{hb−1}−1$.
Следовательно, утверждение верно и для всего дерева.

Теорема. КЧД имеет высоту O(logN)

Рассмотрим красно-чёрное дерево с высотой h. Так как у красной вершины чёрные дети (по свойству 3) количество красных вершин не больше h/2. Тогда чёрных вершин не меньше, чем h/2−1.
По доказанной лемме, для количества внутренних вершин в дереве N выполняется неравенство:
$N⩾2^{h/2}−1$
Прологарифмировав неравенство, имеем:<br>
log(N+1)⩾h2<br>
2log(N+1)⩾h<br>
h⩽2log(N+1)<br>

Добавление

Каждый элемент вставляется вместо листа, поэтому для выбора места вставки идём от корня до тех пор, пока указатель на следующего сына не станет nil (то есть этот сын — лист). Вставляем вместо него новый элемент с нулевыми потомками и красным цветом. Теперь проверяем балансировку. Если отец нового элемента черный, то никакое из свойств дерева не нарушено. Если же он красный, то нарушается свойство 3, для исправления достаточно рассмотреть два случая: 

"Дядя" этого узла тоже красный. Тогда, чтобы сохранить свойства 3 и 4, просто перекрашиваем "отца" и "дядю" в чёрный цвет, а "деда" — в красный. В таком случае черная высота в этом поддереве одинакова для всех листьев и у всех красных вершин "отцы" черные. Проверяем, не нарушена ли балансировка. Если в результате этих перекрашиваний мы дойдём до корня, то в нём в любом случае ставим 

![jupyter](img/lesson_10_add_1.png)


"Дядя" чёрный. Если выполнить только перекрашивание, то может нарушиться постоянство чёрной высоты дерева по всем ветвям. Поэтому выполняем поворот. Если добавляемый узел был правым потомком, то необходимо сначала выполнить левое вращение, которое сделает его левым потомком. Таким образом, свойство 3 и постоянство черной высоты сохраняются.

![jupyter](img/lesson_10_add_2.png)

Удаление

При удалении вершины могут возникнуть три случая в зависимости от количества её детей:
* Если у вершины нет детей, то изменяем указатель на неё у родителя на nil
* Если у неё только один ребёнок, то делаем у родителя ссылку на него вместо этой вершины.
* Если же имеются оба ребёнка, то находим вершину со следующим значением ключа. У такой вершины нет левого ребёнка (так как такая вершина находится в правом поддереве исходной вершины и она самая левая в нем, иначе бы мы взяли ее левого ребенка. Иными словами сначала мы переходим в правое поддерево, а после спускаемся вниз в левое до тех пор, пока у вершины есть левый ребенок). Удаляем уже эту вершину описанным во втором пункте способом, скопировав её ключ в изначальную вершину.

Проверим балансировку дерева. Так как при удалении красной вершины свойства дерева не нарушаются, то восстановление балансировки потребуется только при удалении чёрной. Рассмотрим ребёнка удалённой вершины. 

1. Если брат этого ребёнка красный, то делаем вращение вокруг ребра между отцом и братом, тогда брат становится родителем отца. Красим его в чёрный, а отца — в красный цвет, сохраняя таким образом черную высоту дерева. Хотя все пути по-прежнему содержат одинаковое количество чёрных узлов, сейчас x имеет чёрного брата и красного отца. Таким образом, мы можем перейти к следующему шагу. 


![jupyter](img/lesson_10_delete_1.png)

2. Если брат текущей вершины был чёрным, то получаем три случая:

* Оба ребёнка у брата чёрные. Красим брата в красный цвет и рассматриваем далее отца вершины. Делаем его черным, это не повлияет на количество чёрных узлов на путях, проходящих через b, но добавит один к числу чёрных узлов на путях, проходящих через x, восстанавливая тем самым влиянние удаленного чёрного узла. Таким образом, после удаления вершины черная глубина от отца этой вершины до всех листьев в этом поддереве будет одинаковой.


![jupyter](img/lesson_10_delete_2.png)

* Если у брата правый ребёнок чёрный, а левый красный, то перекрашиваем брата и его левого сына и делаем вращение. Все пути по-прежнему содержат одинаковое количество чёрных узлов, но теперь у x есть чёрный брат с красным правым потомком, и мы переходим к следующему случаю. Ни x, ни его отец не влияют на эту трансформацию. 

![jupyter](img/lesson_10_delete_3.png)

* Если у брата правый ребёнок красный, то перекрашиваем брата в цвет отца, его ребёнка и отца — в чёрный, делаем вращение. Поддерево по-прежнему имеет тот же цвет корня, поэтому свойство 3 и 4 не нарушаются. Но у x теперь появился дополнительный чёрный предок: либо a стал чёрным, или он и был чёрным и b был добавлен в качестве чёрного дедушки. Таким образом, проходящие через x пути проходят через один дополнительный чёрный узел. Выходим из алгоритма. 

![jupyter](img/lesson_10_delete_4.png)

Продолжаем тот же алгоритм, пока текущая вершина чёрная и мы не дошли до корня дерева. Из рассмотренных случаев ясно, что при удалении выполняется не более трёх вращений. 

Объединение

Объединение двух красно-чёрных деревьев T1 и T2 по ключу k возвращает дерево с элементами из T2, T1 и k. Требование: ключ k — разделяющий. То есть ∀k1∈T1,k2∈T2:k1⩽k⩽k2.
Если оба дерева имеют одинаковую черную высоту, то результатом будет дерево с черным корнем k, левым и правым поддеревьями k1 и k2 соответствено.
Теперь пусть у T1
черная высота больше (иначе аналогично).
1. Находим в дереве T1 вершину y на черной высоте, как у дерева T2 вершину с максимальным ключом. Это делается несложно (особенно если мы знаем черную высоту дерева): спускаемся вниз, поддерживая текущую черную высоту.
* Идем вправо. Когда высота станет равной высоте T2, остановимся. Заметим, что черная высота T2⩾2. Поэтому в дереве T1 мы не будем ниже, чем 2. Пусть мы не можем повернуть направо (сын нулевой), тогда наша высота 2 (если мы в черной вершине) или 1 (если в красной). Второго случая быть не может, ибо высота T2⩾2, а в первом случае мы должны были завершить алгоритм, когда пришли в эту вершину. Очевидно, мы окажемся в черной вершине (ибо следующий шаг даст высоту меньше). Очевидно, мы оказались на нужной высоте.     Теперь пусть мы попали не туда. То есть существует путь от корня до другой вершины. Посмотрим на то место, где мы не туда пошли. Если мы пошли вправо, а надо бы влево, то x     имеет больший ключ (по свойству дерева поиска). А если пошли влево, а не вправо, значит правого сына и нет (точнее, есть, но он нулевой), значит в правом поддереве вообще нет вершин. Более того, все вершины с высотами меньше y, которые имеют ключ больше y, будут находиться в поддереве y. Действительно, мы всегда идем вправо. Инвариант алгоритма на каждом шаге — в поддереве текущей вершины содержатся все вершины, ключ которых больше текущего. Проверяется очевидно. Еще поймем, как будем хранить черную высоту дерева. Изначально она нулевая (в пустом дереве). Далее просто поддерживаем ее при операциях вставки и удаления.
2. Объединим поддерево. k будет корнем, левым и правым сыновьями будут Ty и T2 соответственно. Покажем, что свойства дерева поиска не нарушены. Так как все ключи поддерева y не более k и все ключи T2 не менее k, то в новом поддереве с корнем k свойства выполняются. Так как k больше любого ключа из T1, то выполняется и для всего дерева.
3. Красим k в красный цвет. Тогда свойство 4 будет выполнено. Далее поднимаемся вверх, как во вставке операциях, поворотами исправляя нарушение правила 3
4. В конце корень красим в черный, если до этого был красный (это всегда можно сделать, ничего не нарушив). 

Сложность: O(T1.h−T2.h)=O(log(n))

Разрезание

Разрезание дерева по ключу k вернет два дерева, ключи первого меньше k, а второго — не меньше.
Пройдем вниз как во время поиска. Все левые поддеревья вершин пути, корень которых не в пути, будут в первом поддереве. Аналогично правые — в правом. Теперь поднимаемся и последовательно сливаем деревья справа и слева с ответами.
За счет того, что функция join работает за разницу высот, и мы объединяем снизу, то, благодаря телескопическому эффекту на работу времени будут влиять только крайние слагаемые, которые порядка глубины дерева. 

Сложность: O(log(n))

Преимущества красно-чёрных деревьев

* Самое главное преимущество красно-черных деревьев в том, что при вставке выполняется не более O(1)
вращений. Это важно, например, в алгоритме построения динамической выпуклой оболочки. Ещё важно, что примерно половина вставок и удалений произойдут задаром.
* Процедуру балансировки практически всегда можно выполнять параллельно с процедурами поиска, так как алгоритм поиска не зависит от атрибута цвета узлов.
* Сбалансированность этих деревьев хуже, чем у АВЛ, но работа по поддержанию сбалансированности в красно-чёрных деревьях обычно эффективнее. Для балансировки красно-чёрного дерева производится минимальная работа по сравнению с АВЛ-деревьями.
* Использует всего 1 бит дополнительной памяти для хранения цвета вершины. Но на самом деле в современных вычислительных системах память выделяется кратно байтам, поэтому это не является преимуществом относительно, например, АВЛ-дерева, которое хранит 2 бита. Однако есть реализации красно-чёрного дерева, которые хранят значение цвета в бите. Пример — Boost Multiindex. В этой реализации уменьшается потребление памяти красно-чёрным деревом, так как бит цвета хранится не в отдельной переменной, а в одном из указателей узла дерева.

Реализация

In [2]:
#define RED true
#define BLACK false

In [3]:
class RBTreeNode {
public:
    int key;
    int data;
    bool color; // true = red, false = black
    RBTreeNode *left;
    RBTreeNode *right;
    RBTreeNode *parent;
};

In [4]:
class RBTree {
private:
    RBTreeNode *root;
    RBTreeNode *nil;
    int count;
    int invalid;
public:
    RBTree(int invalid);
    ~RBTree();

    void print();
    int getDataByKey(int key);
    void add(int key, int data);
    void remove(int key);
};

In [5]:
#include <iostream>
#include <stack>
#include <string>
using namespace std;

In [23]:
RBTreeNode *newRBTreeNode(int key, int data, bool color = BLACK) {
    auto node = new RBTreeNode;
    node->key = key;
    node->data = data;
    node->left = nullptr;
    node->right = nullptr;
    node->parent = nullptr;
    node->color = color;
    return node;
}

повороты

In [7]:
RBTreeNode* leftRotate(RBTreeNode* root, RBTreeNode** tree_root){
    if (root == nullptr)
        return nullptr;
    if(root->right == nullptr)
        return root->left;

    auto connect = root->parent;
    auto left = (connect != nullptr) and (connect->left == root);
    auto x = root;
    auto y = x->right;
    auto a = x->left;
    auto b = y->left;
    auto c = y->right;

    x->left = a;
    a->parent = x;

    x->right = b;
    b->parent = x;

    y->right = c;
    c->parent = y;

    y->left = x;
    x->parent = y;

    if (connect == nullptr) {
        y->parent = nullptr;
        *tree_root = y;
    }else{
        if (left)
            connect->left = y;
        else
            connect->right = y;
        y->parent = connect;
    }
    return y;
}


In [8]:
RBTreeNode* rightRotate(RBTreeNode* root, RBTreeNode** tree_root){
    if (root == nullptr)
        return nullptr;
    if(root->left == nullptr)
        return root->right;

    auto connect = root->parent;
    auto left = (connect != nullptr) and (connect->left == root);
    auto y = root;
    auto x = y->left;
    auto a = x->left;
    auto b = x->right;
    auto c = y->right;

    y->left = b;
    b->parent = y;

    y->right = c;
    c->parent = y;

    x->left = a;
    a->parent = x;

    x->right = y;
    y->parent = x;

    if (connect == nullptr) {
        x->parent = nullptr;
        *tree_root = x;
    }else{
        if (left)
            connect->left = x;
        else
            connect->right = x;
        x->parent = connect;
    }

    return x;
}

конструктор/деструктор

In [9]:
RBTree::RBTree(int invalid){
    nil = newRBTreeNode(invalid, invalid, BLACK);
    root = nil;
    count = 0;
    this->invalid = invalid;
}

In [10]:
RBTree::~RBTree(){
    count = 0;

    stack<RBTreeNode*> stack;
    stack.push(root);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nil){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }
    delete nil;
}

печать

In [11]:
void printTree(RBTreeNode *root, string indent, bool last) {
    if (root != nullptr) {
        cout << indent;
        if (last) {
            cout << "R----";
            indent += "   ";
        } else {
            cout << "L----";
            indent += "|  ";
        }
        cout << "( key: " <<root->key << ", data: " << root->data << ", color: " << root->color <<")" << endl;
        printTree(root->left, indent, false);
        printTree(root->right, indent, true);
    }
}

In [12]:
void RBTree::print(){
    printTree(root, "", true);
}

поиск

In [13]:
RBTreeNode* search(RBTreeNode* root, int key, RBTreeNode* nil){
    if (root == nil || root->key == key)
        return root;
    if (key < root->key)
        return search(root->left, key, nil);
    return search(root->right, key, nil);
}

In [15]:
int RBTree::getDataByKey(int key){
    auto node = search(root, key, nil);
    if (node != nil)
        return node->data;
    return invalid;
}

input_line_22:1:13: error: redefinition of 'getDataByKey'
int RBTree::getDataByKey(int key){
            ^
input_line_21:1:13: note: previous definition is here
int RBTree::getDataByKey(int key){
            ^


Interpreter Error: 

добавление

In [16]:
RBTreeNode* goDown(RBTreeNode* root, int key, RBTreeNode* nil){
    RBTreeNode* node = root;
    RBTreeNode* parent = nullptr;
    while ((node != nil) and (node->key != key)){
        parent = node;
        if (key < node->key)
            node = node->left;
        else
            node = node->right;
    }
    return parent;
}


In [17]:
void fixAdd(RBTreeNode*& root, RBTreeNode* node) {
    if(root == node){
        root->color = BLACK;
        return;
    }
    while ((node->parent != nullptr) && (node->parent->color == RED)){
        auto parent = node->parent;
        auto grandfather = parent->parent;

        if (parent == grandfather->left){ //отец левый ребенок
            auto uncle = grandfather->right;
            if (uncle->color == RED){ // есть красный дядя
                parent->color = BLACK;
                uncle->color = BLACK;
                grandfather->color = RED;
                node = parent->parent;
            } else { // нет красного дяди
                if (node == parent->right){ //доворот
                    node = parent;
                    leftRotate(node, &root);
                }
                parent->color = BLACK;
                grandfather->color = RED;
                rightRotate(grandfather, &root);
            }
        } else { // отец правый ребенок
            auto uncle = grandfather->left;
            if (uncle->color == RED){ // есть красный дядя
                parent->color = BLACK;
                uncle->color = BLACK;
                grandfather->color = RED;
                node = parent->parent;
            } else { // нет красного дяди
                if (node == parent->left){ //доворот
                    node = parent;
                    rightRotate(node, &root);
                }
                parent->color = BLACK;
                grandfather->color = RED;
                leftRotate(grandfather, &root);
            }
        }
    }
    root->color = BLACK;
}

In [24]:
void RBTree::add(int key, int data) {
    auto parent = goDown(root, key, nil);

    if(parent == nullptr) {
        root = newRBTreeNode(key, data, BLACK);
        root->left = root->right = nil;
    } else if (parent->key == key){ // обновление данных
        parent->data = data;
    } else {
        auto node = newRBTreeNode(key, data, RED);
        node->left = node->right = nil;
        node->parent = parent;
        if (key < parent->key)
            parent->left = node;
        else
            parent->right = node;
        fixAdd(root, node);
    }
}


input_line_31:1:14: error: redefinition of 'add'
void RBTree::add(int key, int data) {
             ^
input_line_25:1:14: note: previous definition is here
void RBTree::add(int key, int data) {
             ^


Interpreter Error: 

удаление

In [19]:
RBTreeNode* findNext(RBTreeNode* root, RBTreeNode* nil){
    RBTreeNode* node = root->right;
    RBTreeNode* parent = root;
    while (node != nil){
        parent = node;
        node = node->left;
    }
    return parent;
}

In [21]:
RBTreeNode* fixRemove(RBTreeNode* root, RBTreeNode* node){
    while ((node->color == BLACK) and (node != root)) {
        auto parent = node->parent;
        if (node == parent->left) { // узел левый ребенок
            auto brother = parent->right;
            if (brother->color == RED) { // брат красный
                brother->color = BLACK;
                parent->color = RED;
                leftRotate(parent, &root);
            }
            if ((brother->left->color == BLACK) and (brother->right->color == BLACK)) { //у брата черные дети
                brother->color = RED;
                node = node->parent;
            } else {
                if (brother->right->color == BLACK) { // брат красный с черным правым ребенком
                    brother->left->color = BLACK;
                    brother->color = RED;
                    rightRotate(brother, &root);
                    //brother = node->parent->right;
                }
                brother->color = parent->color;
                parent->color = BLACK;
                brother->right->color = BLACK;
                leftRotate(parent, &root);
                node = root; // условие завершения цикла
            }
        } else { // узел правый ребенок
            auto brother = parent->left;
            if (brother->color == RED) { // брат красный
                brother->color = BLACK;
                parent->color = RED;
                rightRotate(parent, &root);
            }
            if ((brother->left->color == BLACK) and (brother->right->color == BLACK)) { //у брата черные дети
                brother->color = RED;
                node = node->parent;
            } else {
                if (brother->left->color == BLACK) { // брат красный с черным правым ребенком
                    brother->right->color = BLACK;
                    brother->color = RED;
                    leftRotate(brother, &root);                    
                }
                brother->color = parent->color;
                parent->color = BLACK;
                brother->left->color = BLACK;
                rightRotate(parent, &root);
                node = root; // условие завершения цикла
            }
        }
    }
    node->color = BLACK;
    root->color = BLACK;
    return root;
}

In [22]:
void RBTree::remove(int key) {
    RBTreeNode* real = nil;
    auto node = search(root, key, nil);

    if((node->left == nil) or (node->right == nil)) // 1 сын
        real = node;
    else
        real = findNext(node, nil);

    // дважды черная
    RBTreeNode* tmp = real->left != nil ? real->left : real->right;
    tmp->parent = real->parent;

    if (real->parent == nil) // корень
        root = tmp;
    else if (real == real->parent->left)
        real->parent->left = tmp;
    else if (real == real->parent->right)
        real->parent->right = tmp;

    if (real != node){
        node->key = real->key;
        node->data = real->data;
    }
    if(real->color == BLACK)
        root = fixRemove(root, tmp);

    delete real;

}


## Декартово дерево

Дерамида, курево, дуча

!!!ОСТОРОЖНО КУРЕВО!!!

In [1]:
struct CartesianTreeNode {
    int key;
    int factor;
    int data;
    CartesianTreeNode *left, *right;
};

In [2]:
class CartesianTree {
private:

    CartesianTreeNode *root;
    int count;
    int invalid;
public:
    CartesianTree(int invalidData);
    ~CartesianTree();

    void print();
    CartesianTree split(int key);
    void merge(CartesianTree* tree1, CartesianTree* tree2);
    int getDataByKey(int key);
    void add(int key, int data);
    void remove(int key);
};


In [3]:
#include <cstdlib>
#include <stack>
#include <iostream>

using namespace std;

In [4]:
CartesianTreeNode *newCatresianTreeNode(int key, int data, int factor = rand()) {
    auto *temp = new CartesianTreeNode;
    temp->key = key;
    temp->data = data;
    temp->factor = factor;
    temp->left = temp->right = nullptr;
    return temp;
}

In [5]:
CartesianTree::CartesianTree(int invalidData){
    root = nullptr;
    count = 0;
    invalid = invalidData;
}

In [6]:
CartesianTree::~CartesianTree(){
    count = 0;

    stack<CartesianTreeNode*> stack;
    stack.push(root);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }
}

In [7]:
void inorder(CartesianTreeNode *root) {
    if (root != nullptr) {
        inorder(root->left);
        cout << root->key << " -> ";
        inorder(root->right);
    }
}

In [8]:
void CartesianTree::print(){
    inorder(root);
}

In [9]:
void split(CartesianTreeNode* node, int key, CartesianTreeNode*& left, CartesianTreeNode*& right){
    if (node == nullptr) {
        left = right = nullptr;
        return;
    }
    CartesianTreeNode *l, *r;
    if(node->key < key) {
        split(node->right, key, l, r);
        node->right = l;
        left = node;
        right = r;
    } else {
        split(node->left, key, l, r);
        node->left = r;
        left = l;
        right = node;
    }
}

In [10]:
CartesianTree CartesianTree::split(int key)
{
    CartesianTree result(invalid);
    CartesianTreeNode *left, *right;
    ::split(root, key, left, right);
    root = left;
    result.root = right;
    return result;
}

In [11]:
CartesianTreeNode* merge(CartesianTreeNode* tree1, CartesianTreeNode* tree2){
    if (tree1 == nullptr)
        return tree2;
    if (tree2 == nullptr)
        return tree1;
    if (tree1->factor > tree2->factor){
        tree1->right = merge(tree1->right, tree2);
        return tree1;
    } else {
        tree2->left = merge(tree1, tree2->left);
        return tree2;
    }
}

In [ ]:
void CartesianTree::merge(CartesianTree* tree1, CartesianTree* tree2){
    root = ::merge(tree1->root, tree2->root);
    tree2->root = nullptr;
}

In [12]:
CartesianTreeNode* search(CartesianTreeNode* root, int key){
    if (root == nullptr || root->key == key)
        return root;
    if (root->key < key)
        return search(root->right, key);
    return search(root->left, key);
}

In [13]:
int CartesianTree::getDataByKey(int key){
    auto node = search(root, key);
    if (node != nullptr)
        return node->data;
    return invalid;
}

In [14]:
CartesianTreeNode* add(CartesianTreeNode* tree, CartesianTreeNode* x){
    if (tree == nullptr)
        return x;
    if (x == nullptr)
        return tree;
    if (x->factor > tree->factor){
        //новый корень
        CartesianTreeNode *left, *right;
        split(tree, x->key, left, right);
        x->left = left;
        x->right = right;
        return x;
    }
    if (x->key < tree->key)
        tree->left = add(tree->left, x);
    else
        tree->right = add(tree->right, x);
    return tree;
}

In [15]:
void CartesianTree::add(int key, int data){
    CartesianTreeNode* node = newCatresianTreeNode(key, data);
    root = ::add(root, node);
}

In [16]:
CartesianTreeNode* remove(CartesianTreeNode* tree, int key){
    if (tree->key == key){
        CartesianTreeNode* left = tree->left;
        CartesianTreeNode* right = tree->right;
        delete tree;
        return merge(left, right);
    }
    if (key < tree->key) {
        tree->left = remove(tree->left, key);
    } else {
        tree->right = remove(tree->right, key);
    }
    return tree;
}

In [17]:
void CartesianTree::remove(int key){
    root = ::remove(root, key);
}

## Декартово дерево по неявному ключу

In [18]:
struct ImplicitKeyTreapNode {
    int count;
    int factor;
    int data;
    ImplicitKeyTreapNode *left, *right;
};

In [19]:
class ImplicitKeyTreap {
private:

    ImplicitKeyTreapNode *root;
    int count;
    int invalid;
public:
    ImplicitKeyTreap(int invalidData);
    ~ImplicitKeyTreap();

    void print();
    ImplicitKeyTreap split(int index);
    void merge(ImplicitKeyTreap* tree1, ImplicitKeyTreap* tree2);
    int at(int index);
    void add(int index, int data);
    void remove(int index);
};

In [20]:
#include <cstdlib>
#include <stack>
#include <iostream>

using namespace std;

In [21]:
ImplicitKeyTreapNode *newImplicitTreeNodeNode(int data, int factor = rand()) {
    auto *temp = new ImplicitKeyTreapNode;
    temp->count = 1;
    temp->data = data;
    temp->factor = factor;
    temp->left = temp->right = nullptr;
    return temp;
}

In [22]:
ImplicitKeyTreap::ImplicitKeyTreap(int invalidData){
    root = nullptr;
    count = 0;
    invalid = invalidData;
}

In [23]:
ImplicitKeyTreap::~ImplicitKeyTreap(){
    count = 0;

    stack<ImplicitKeyTreapNode*> stack;
    stack.push(root);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }
}

In [24]:
void inorder(ImplicitKeyTreapNode *root) {
    if (root != nullptr) {
        inorder(root->left);
        cout << root->data << " -> ";
        //cout << root->count << " -> ";
        inorder(root->right);
    }
}

In [25]:
void ImplicitKeyTreap::print(){
    inorder(root);
}

In [26]:
ImplicitKeyTreapNode* search(ImplicitKeyTreapNode* root, int index){
    if ((root == nullptr) or (index < 0))
        return nullptr;
    if (root->left == nullptr)
        if (index == 0)
            return root;
        else
            return search(root->right, index - 1);
    else
        if (index - root->left->count == 0)
            return root;
        else
            if (index < root->left->count)
                return search(root->left, index);
            else
                return search(root->right, index - root->left->count - 1);
}

In [27]:
int ImplicitKeyTreap::at(int index){
    ImplicitKeyTreapNode* node = ::search(root, index);
    if (node == nullptr)
        return invalid;
    else
        return node->data;
}

In [28]:
int recount(ImplicitKeyTreapNode* node){
    if (node == nullptr)
        return 0;
    int cnt = (node->left == nullptr) ? 0 : node->left->count;
    cnt += (node->right == nullptr) ? 0 : node->right->count;
    return cnt;
}

In [29]:
int count(ImplicitKeyTreapNode* node){
    if (node == nullptr)
        return 0;
    return node->count;
}

In [30]:
void split(ImplicitKeyTreapNode* node, int index, ImplicitKeyTreapNode*& left, ImplicitKeyTreapNode*& right){
    if (node == nullptr) {
        left = right = nullptr;
        return;
    }
    ImplicitKeyTreapNode *l, *r;
    if(index <= count(node->left)) {
        split(node->left, index, l, r);
        node->left = r;
        node->count = recount(node) + 1;
        left = l;
        right = node;
    } else {
        split(node->right, index - count(node->left) - 1, l, r);
        node->right = l;
        node->count = recount(node) + 1;
        left = node;
        right = r;
    }
}

In [31]:
ImplicitKeyTreap ImplicitKeyTreap::split(int index){
    ImplicitKeyTreap result(invalid);
    ImplicitKeyTreapNode *left, *right;
    ::split(root, index, left, right);
    root = left;
    result.root = right;
    return result;
}

In [32]:
ImplicitKeyTreapNode* merge(ImplicitKeyTreapNode* tree1, ImplicitKeyTreapNode* tree2){
    if (tree1 == nullptr)
        return tree2;
    if (tree2 == nullptr)
        return tree1;
    if (tree1->factor > tree2->factor){
        tree1->right = merge(tree1->right, tree2);
        tree1->count = recount(tree1);
        return tree1;
    } else {
        tree2->left = merge(tree1, tree2->left);
        tree2->count = recount(tree1);
        return tree2;
    }
}

In [33]:
void ImplicitKeyTreap::merge(ImplicitKeyTreap* tree1, ImplicitKeyTreap* tree2){
    root = ::merge(tree1->root, tree2->root);
    tree2->root = nullptr;
}

In [34]:
ImplicitKeyTreapNode* add(ImplicitKeyTreapNode* tree, ImplicitKeyTreapNode* x, int index){
    if (tree == nullptr)
        return x;
    if (x == nullptr)
        return tree;
    if (x->factor > tree->factor){
        //новый корень
        ImplicitKeyTreapNode *left, *right;
        split(tree, index, left, right);
        x->left = left;
        x->right = right;
        x->count = recount(x) + 1;
        return x;
    }

    if (index <= count(tree->left)) {
        tree->left = add(tree->left, x, index);
        tree->left->count = recount(tree->left) + 1;
    } else {
        tree->right = add(tree->right, x, index - count(tree->left) - 1);
        tree->right->count = recount(tree->right) + 1;
    }
    return tree;
}

In [35]:
void ImplicitKeyTreap::add(int index, int data){
    ImplicitKeyTreapNode* node = newImplicitTreeNodeNode(data);
    root = ::add(root, node, index);
    root->count = ::recount(root) + 1;
}

In [36]:
ImplicitKeyTreapNode* remove(ImplicitKeyTreapNode* tree, int index){
    if (count(tree->left) == index){
        ImplicitKeyTreapNode* left = tree->left;
        ImplicitKeyTreapNode* right = tree->right;
        delete tree;
        ImplicitKeyTreapNode* tmp = merge(left, right);
        return tmp;
    }
    if (index < count(tree->left))
        if (tree->left != nullptr) {
            tree->left = remove(tree->left, index);
            tree->left->count = recount(tree->left);
        }
    else
        if (tree->right != nullptr){
            tree->right = remove(tree->right, index);
            tree->right->count = recount(tree->right);
        }
    return tree;
}

input_line_44:14:5: warning: add explicit braces to avoid dangling else [-Wdangling-else]
    else
    ^


In [37]:
void ImplicitKeyTreap::remove(int index){
    root = ::remove(root, index);
}


# Небинарные деревья поиска

## B-дерево

Структура B-дерева применяется для организации индексов во многих современных СУБД.

B-дерево может применяться для структурирования (индексирования) информации на жёстком диске (как правило, метаданных). Время доступа к произвольному блоку на жёстком диске очень велико (порядка миллисекунд), поскольку оно определяется скоростью вращения диска и перемещения головок. Поэтому важно уменьшить количество узлов, просматриваемых при каждой операции. Использование поиска по списку каждый раз для нахождения случайного блока могло бы привести к чрезмерному количеству обращений к диску вследствие необходимости последовательного прохода по всем его элементам, предшествующим заданному, тогда как поиск в B-дереве, благодаря свойствам сбалансированности и высокой ветвистости, позволяет значительно сократить количество таких операций.

Относительно простая реализация алгоритмов и существование готовых библиотек (в том числе для C) для работы со структурой B-дерева обеспечивают популярность применения такой организации памяти в самых разнообразных программах, работающих с большими объёмами данных. 

B-дерево является идеально сбалансированным, то есть глубина всех его листьев одинакова. B-дерево имеет следующие свойства (t — параметр дерева, называемый минимальной степенью B-дерева, не меньший 2.): 
* Каждый узел, кроме корня, содержит не менее t−1 ключей, и каждый внутренний узел имеет по меньшей мере t дочерних узлов. Если дерево не является пустым, корень должен содержать как минимум один ключ.
* Каждый узел, кроме корня, содержит не более 2t−1 ключей и не более чем 2t сыновей во внутренних узлах
* Корень содержит от 1 до 2t−1 ключей, если дерево не пусто и от 2 до 2t детей при высоте большей 0 .
* Каждый узел дерева, кроме листьев, содержащий ключи k1,...,kn, имеет n+1 сына. i-й сын содержит ключи из отрезка $[k_{i−1};k_i]$, $k_0=-\infty,k_{n+1}=\infty$
.
Ключи в каждом узле упорядочены по неубыванию.
Все листья находятся на одном уровне.

Основные достоинства
* Во всех случаях полезное использование пространства вторичной памяти составляет свыше 50 %. С ростом степени полезного использования памяти не происходит снижения качества обслуживания.
* Произвольный доступ к записи реализуется посредством малого количества подопераций (обращения к физическим блокам).
* В среднем достаточно эффективно реализуются операции включения и удаления записей; при этом сохраняется естественный порядок ключей с целью последовательной обработки, а также соответствующий баланс дерева для обеспечения быстрой произвольной выборки.
* Неизменная упорядоченность по ключу обеспечивает возможность эффективной пакетной обработки.

Основной недостаток В-деревьев состоит в отсутствии для них эффективных средств выборки данных (то есть метода обхода дерева), упорядоченных по свойству, отличному от выбранного ключа. 

## 2-3 Дерево

2-3 дерево (англ. 2-3 tree) — структура данных, представляющая собой сбалансированное дерево поиска, такое что из каждого узла может выходить две или три ветви и глубина всех листьев одинакова. Является частным случаем B+ дерева. 

Свойства:
* нелистовые вершины имеют либо 2, либо 3 сына 
* нелистовая вершина, имеющая двух сыновей, хранит максимум левого поддерева. Нелистовая вершина, имеющая трех сыновей, хранит два значения. Первое значение хранит максимум левого поддерева, второе максимум центрального поддерева
* сыновья упорядочены по значению максимума поддерева сына
* все листья лежат на одной глубине
* высота 2-3 дерева O(logn), где n — количество элементов в дереве.


## Splay дерево

In [20]:
class SplayTreeNode {
public:
    int key;
    int data;
    SplayTreeNode *left;
    SplayTreeNode *right;
};


In [21]:
class SplayTree {
private:
    SplayTreeNode *root;
    int count;
    int invalid;
public:
    SplayTree(int invalid);
    ~SplayTree();

    void print();
    int getDataByKey(int key);
    void add(int key, int data);
    void remove(int key);
};


In [22]:
#include <iostream>
#include <stack>
#include <string>
using namespace std;

In [23]:
SplayTreeNode *newSplayTreeNode(int key, int data) {
    auto node = new SplayTreeNode;
    node->key = key;
    node->data = data;
    node->left = nullptr;
    node->right = nullptr;
    return node;
}

In [24]:
SplayTree::SplayTree(int invalid){
    root = nullptr;
    count = 0;
    this->invalid = invalid;
}

In [25]:
SplayTree::~SplayTree(){
    count = 0;

    stack<SplayTreeNode*> stack;
    stack.push(root);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }
}

In [26]:
void printTree(SplayTreeNode *root, string indent, bool last) {
    if (root != nullptr) {
        cout << indent;
        if (last) {
            cout << "R----";
            indent += "   ";
        } else {
            cout << "L----";
            indent += "|  ";
        }
        cout << "(" <<root->key << ", " << root->data << ")" << endl;
        printTree(root->left, indent, false);
        printTree(root->right, indent, true);
    }
}

In [27]:
void SplayTree::print(){
    printTree(root, "", true);
}

In [28]:
SplayTreeNode *rightRotate(SplayTreeNode *x)
{
    SplayTreeNode *y = x->left;
    x->left = y->right;
    y->right = x;
    return y;
}

In [29]:
SplayTreeNode *leftRotate(SplayTreeNode *x)
{
    SplayTreeNode *y = x->right;
    x->right = y->left;
    y->left = x;
    return y;
}

In [30]:
SplayTreeNode *splay(SplayTreeNode *&root, int key)
{
    if (root == nullptr || root->key == key)
        return root;

    // Ключ лежит в левом поддереве
    if (root->key > key)
    {
        // Ключа нет в дереве, мы закончили
        if (root->left == nullptr) return root;

        // Zig-Zig (Левый-левый)
        if (root->left->key > key)
        {
            // Сначала рекурсивно поднимем
            // ключ как корень left-left
            root->left->left = splay(root->left->left, key);

            // Первый разворот для root,
            // второй разворот выполняется после else
            root = rightRotate(root);
        }
        else if (root->left->key < key) // Zig-Zag (Левый-правый)
        {
            // Сначала рекурсивно поднимаем
            // ключ как корень left-right
            root->left->right = splay(root->left->right, key);

            // Выполняем первый разворот для root->left
            if (root->left->right != nullptr)
                root->left = leftRotate(root->left);
        }

        // Выполняем второй разворот для корня
        return (root->left == nullptr)? root: rightRotate(root);
    }
    else // Ключ находится в правом поддереве
    {
        // Ключа нет в дереве, мы закончили
        if (root->right == nullptr) return root;

        // Zag-Zig (Правый-левый)
        if (root->right->key > key)
        {
            // Поднять ключ как корень right-left
            root->right->left = splay(root->right->left, key);

            // Выполняем первый поворот для root->right
            if (root->right->left != nullptr)
                root->right = rightRotate(root->right);
        }
        else if (root->right->key < key)// Zag-Zag (Правый-правый)
        {
            // Поднимаем ключ как корень
            // right-right и выполняем первый разворот
            root->right->right = splay(root->right->right, key);
            root = leftRotate(root);
        }

        // Выполняем второй разворот для root
        return (root->right == nullptr)? root: leftRotate(root);
    }
}

In [31]:
int SplayTree::getDataByKey(int key){
    root = splay(root, key);;
    if (root != nullptr)
        return root->data;
    return invalid;
}

In [32]:
SplayTreeNode* insert(SplayTreeNode *node, int key, int data) {
    if (node == nullptr) return newSplayTreeNode(key, data);

    if (key < node->key)
        node->left = insert(node->left, key, data);
    else
        node->right = insert(node->right, key, data);

    return node;
}

In [33]:
void SplayTree::add(int key, int data){
    root = ::insert(root, key, data);
}

In [34]:
SplayTreeNode* deleteNode(SplayTreeNode* root, int key){
    if (root == nullptr) return root;

    if (key < root->key)
        root->left = deleteNode(root->left, key);
    else if (key > root->key)
        root->right = deleteNode(root->right, key);
    else {
        if (root->left == nullptr) {
            auto temp = root->right;
            delete root;
            return temp;
        } else if (root->right == nullptr) {
            auto  *temp = root->left;
            delete root;
            return temp;
        }

        //если узел в корне надо в правом дереве найти минимум и поставить в корень
        auto prnt = root;
        auto curr = root->right;
        while (curr->left != nullptr) {
            prnt = curr;
            curr = curr->left;
        }
        root->key = curr->key;
        root->data = curr->data;
        root->right = deleteNode(root->right, curr->key);
    }
    return root;
}


In [35]:
void SplayTree::remove(int key){
    root = ::deleteNode(root, key);
}

## Scapegoat дерево

In [5]:
#include <cmath>
#include <iostream>
#include <stack>

In [6]:
using namespace std;

In [1]:
struct ScapegoatTreeNode{

    ScapegoatTreeNode *left, *right, *parent;
    int key;
    int data;
    int size;

    ScapegoatTreeNode(){
        key = data = size = 0;
        left = right = parent = nullptr;
    }

    ScapegoatTreeNode(int key, int data, int size = 1){
        this->key = key;
        this->data = data;
        this->size = size;
        left = right = parent = nullptr;
    }
};


In [2]:
class ScapegoatTree
{
private:
    ScapegoatTreeNode *root;
    int count;  // Number of nodes in Tree
    int invalid;


    bool isEmpty();
    void makeEmpty();
    int size();
    float alpha;

    int recountSize(ScapegoatTreeNode* root);
    //void rebuild(ScapegoatTreeNode *&root);
public:
    ScapegoatTree(float alpha, int invalid);
    ~ScapegoatTree();

    void print();
    int getDataByKey(int key);
    void add(int key, int data);
    void remove(int key);

};

In [3]:
ScapegoatTree::ScapegoatTree(float alpha, int invalid){
    root = nullptr;
    this->alpha = alpha;
    this->invalid = invalid;
    count = 0;
}

In [4]:
ScapegoatTree::~ScapegoatTree(){
    count = 0;

    stack<ScapegoatTreeNode*> stack;
    stack.push(root);

    while (true){
        if (stack.empty())
            break;

        auto curr = stack.top(); stack.pop();

        if (curr != nullptr){
            stack.push(curr->left);
            stack.push(curr->right);
            delete curr;
        }
    }
}

input_line_10:3:5: error: use of undeclared identifier 'stack'
    stack<ScapegoatTreeNode*> stack;
    ^
input_line_10:3:11: error: 'ScapegoatTreeNode' does not refer to a value
    stack<ScapegoatTreeNode*> stack;
          ^
input_line_7:1:8: note: declared here
struct ScapegoatTreeNode{
       ^
input_line_10:3:29: error: expected expression
    stack<ScapegoatTreeNode*> stack;
                            ^
input_line_10:3:31: error: use of undeclared identifier 'stack'
    stack<ScapegoatTreeNode*> stack;
                              ^
input_line_10:4:5: error: use of undeclared identifier 'stack'
    stack.push(root);
    ^
input_line_10:6:13: error: use of undeclared identifier 'stack'
        if (stack.empty())
            ^
input_line_10:8:21: error: use of undeclared identifier 'stack'
        auto curr = stack.top(); stack.pop();
                    ^
input_line_10:8:34: error: use of undeclared identifier 'stack'
        auto curr = stack.top(); stack.pop();
              

Interpreter Error: 

In [7]:
void printTree(ScapegoatTreeNode *root, string indent, bool last) {
    if (root != nullptr) {
        cout << indent;
        if (last) {
            cout << "R----";
            indent += "   ";
        } else {
            cout << "L----";
            indent += "|  ";
        }
        cout << "(" <<root->key << ", " << root->data << ")" << endl;
        printTree(root->left, indent, false);
        printTree(root->right, indent, true);
    }
}

In [8]:
void ScapegoatTree::print(){
    printTree(root, "", true);
}

In [9]:
ScapegoatTreeNode* search(ScapegoatTreeNode* root, int key){
    if (root == nullptr || root->key == key)
        return root;
    if (root->key < key)
        return search(root->right, key);
    return search(root->left, key);
}

In [10]:
int ScapegoatTree::getDataByKey(int key){
    auto node = search(root, key);
    if (node != nullptr)
        return node->data;
    return invalid;
}

In [11]:
ScapegoatTreeNode* addWithSize(ScapegoatTreeNode *root, ScapegoatTreeNode *&node, int key, int data) {
    if (root == nullptr){
        node = new ScapegoatTreeNode(key, data, 1);
        return node;
    }
    if (key < root->key) {
        root->left = addWithSize(root->left, node, key, data);
        root->left->parent = root;
        root->size++;
    }else{
        root->right = addWithSize(root->right, node, key, data);
        root->right->parent = root;
        root->size++;
    }
    return root;
}

In [12]:
int nodesize(ScapegoatTreeNode* node){
    if (node == nullptr)
        return 0;
    return node->size;
}

In [13]:
ScapegoatTreeNode* findHighestDisbalance(ScapegoatTreeNode* node, float factor){
    ScapegoatTreeNode* res = nullptr;
    while (node != nullptr){
        if ((nodesize(node->left) > nodesize(node) * factor) || (nodesize(node->right) > nodesize(node) * factor))
            res = node;
        node = node->parent;
    }
    return res;
}

In [14]:
int packIntoArray(ScapegoatTreeNode* root, ScapegoatTreeNode* result[], int index) {
    if (root == nullptr) {
        return index;
    }
    index = packIntoArray(root->left, result, index);
    result[index++] = root;
    return packIntoArray(root->right, result, index);
}

In [15]:
ScapegoatTreeNode* buildBalanced(ScapegoatTreeNode* tree_in_array[], int index, int size) {
    if (size == 0)
        return nullptr;
    int m = size / 2;
    tree_in_array[index + m]->left = buildBalanced(tree_in_array, index, m);
    if (tree_in_array[index + m]->left != nullptr)
        tree_in_array[index + m]->left->parent = tree_in_array[index + m];
    tree_in_array[index + m]->right = buildBalanced(tree_in_array, index + m + 1, size - m - 1);
    if (tree_in_array[index + m]->right != nullptr)
        tree_in_array[index + m]->right->parent = tree_in_array[index + m];
    tree_in_array[index + m]->size = size;
    return tree_in_array[index + m];
}

In [16]:
void rebuild(ScapegoatTreeNode *&root) {
    if (root == nullptr)
        return;
    int size = root->size;
    ScapegoatTreeNode* parent = root->parent;
    ScapegoatTreeNode** array = new ScapegoatTreeNode*[size];
    packIntoArray(root, array, 0);
    if (parent == nullptr) {
        root = buildBalanced(array, 0, size);
        root->parent = nullptr;
    } else if (parent->right == root) {
        parent->right = buildBalanced(array, 0, size);
        parent->right->parent = parent;
    } else {
        parent->left = buildBalanced(array, 0, size);
        parent->left->parent = parent;
    }
    delete[] array;
}

In [17]:
void ScapegoatTree::add(int key, int data){
    ScapegoatTreeNode* node; // узел для начала подъема
    root = addWithSize(root, node, key, data);
    auto disbalanced = findHighestDisbalance(node, alpha);
    if (disbalanced == root)
        rebuild(root);
    else
        rebuild(disbalanced);
}

In [18]:
ScapegoatTreeNode* deleteNode(ScapegoatTreeNode* root, ScapegoatTreeNode *&parent, int key){
    if (root == nullptr) return root;

    root->size--;
    if (key < root->key) {
        root->left = deleteNode(root->left, parent, key);
    } else if (key > root->key) {
        root->right = deleteNode(root->right, parent, key);
    } else {
        if ((root->left == nullptr) && (root->right == nullptr)) {
            //auto temp = root->right;
            parent = root->parent;
            if (parent->left == root)
                parent->left = nullptr;
            else
                parent->right = nullptr;
            delete root;
            return nullptr;
        }

        auto prnt = root;
        auto curr = root;

        if (root->right != nullptr){ //если узел в корне надо в правом дереве найти минимум и поставить в корень

            curr = root->right;
            while (curr->left != nullptr) {
                prnt = curr;
                curr = curr->left;
            }

        }else{ //если узел в корне надо в левом дереве найти максимум и поставить в корень

            curr = root->left;
            while (curr->right != nullptr) {
                prnt = curr;
                curr = curr->right;
            }
        }
        root->key = curr->key;
        root->data = curr->data;
        parent = prnt;
        if (parent->right == curr)
            parent->right = nullptr;
        else
            parent->left = nullptr;
        delete curr;

    }
    return root;
}

In [19]:
void ScapegoatTree::remove(int key){
    ScapegoatTreeNode* node;
    root = ::deleteNode(root, node, key);
    auto disbalanced = findHighestDisbalance(node, alpha);
    if (disbalanced == root)
        rebuild(root);
    else
        rebuild(disbalanced);
}